In [1]:
begin
	import Pkg
    Pkg.activate()
	#Ojo, dado que activamos Pkg, ya no estamos en un entorno de notebooks reproducibles.
end

  Activating project at `~/.julia/environments/v1.11`


In [2]:
begin
	using DataFrames
	using Dates
	using CategoricalArrays
	using DuckDB
	using CSV
	using MLJ, MLJBase
	using MLJModels
	using Statistics, Distributions;	

end

In [3]:
begin
    data = CSV.read("../dataset/bank-full.csv", DataFrame)
end

Row,age,job,marital,education,default,balance,housing,loan,contact,day,month,durations,campaign,pdays,previous,poutcome,y
,Int64,String15,String15,String15,String3,Int64,String3,String3,String15,Int64,String3,Int64,Int64,Int64,Int64,String7,String3
1,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
2,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
3,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
4,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
5,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
6,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
7,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
8,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
9,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no



---

# Creating Aux feature for correct stratification

In [4]:
# Create stratification variable by combining both features
data.strat_var = coerce(string.(data.default, "_", data.y), Finite)

#strat_var = coerce(data.strat_var, Finite)

# Stratified split
train_indices, test_indices = partition(
    eachindex(data.strat_var), 
    0.8, 
    stratify=data.strat_var,
    rng= 37  # for reproducibility
)

train_data = data[train_indices, :]
test_data = data[test_indices, :]

# Remove the temporary stratification column
select!(train_data, Not(:strat_var))
select!(test_data, Not(:strat_var))

Row,age,job,marital,education,default,balance,housing,loan,contact,day,month,durations,campaign,pdays,previous,poutcome,y
,Int64,String15,String15,String15,String3,Int64,String3,String3,String15,Int64,String3,Int64,Int64,Int64,Int64,String7,String3
1,40,admin.,divorced,secondary,no,2550,yes,no,unknown,2,jun,424,2,-1,0,unknown,no
2,40,blue-collar,married,secondary,no,563,yes,no,unknown,7,may,615,1,-1,0,unknown,no
3,29,technician,single,secondary,no,706,yes,no,cellular,18,may,1340,1,-1,0,unknown,yes
4,44,management,married,tertiary,no,119,yes,no,cellular,10,jul,67,1,-1,0,unknown,no
5,27,blue-collar,single,secondary,no,185,yes,no,cellular,7,may,212,1,-1,0,unknown,no
6,43,blue-collar,married,secondary,no,19,yes,no,unknown,16,may,249,1,-1,0,unknown,no
7,39,admin.,married,secondary,no,201,yes,no,cellular,13,apr,263,3,-1,0,unknown,no
8,47,unemployed,married,secondary,no,351,yes,yes,cellular,19,nov,174,1,174,1,failure,no
9,38,management,married,tertiary,no,4293,yes,no,cellular,6,may,92,2,250,2,failure,no


Check crosstabs

In [5]:
using FreqTables

# Create a crosstab table for default and y
crosstab_table = freqtable(train_data.default, train_data.y)
println("Crosstab table for default and y:")
println(crosstab_table)

# Convert to DataFrame for better display
# Need to convert the NamedMatrix to a regular matrix first
crosstab_matrix = Matrix(crosstab_table)
row_names = names(crosstab_table, 1)
col_names = names(crosstab_table, 2)

# Create DataFrame with proper column names
crosstab_df = DataFrame(crosstab_matrix, col_names)
# Add row names as the first column
insertcols!(crosstab_df, 1, :default => row_names)

println("\nAs DataFrame:")
println(crosstab_df)

Crosstab table for default and y:
[31326 4190; 610 42]

As DataFrame:
2×3 DataFrame
 Row │ default  no     yes   
     │ String3  Int64  Int64 
─────┼───────────────────────
   1 │ no       31326   4190
   2 │ yes        610     42


In [6]:
# Create a crosstab table for default and y
crosstab_table = freqtable(test_data.default, test_data.y)
println("Crosstab table for default and y:")
println(crosstab_table)

# Convert to DataFrame for better display
# Need to convert the NamedMatrix to a regular matrix first
crosstab_matrix = Matrix(crosstab_table)
row_names = names(crosstab_table, 1)
col_names = names(crosstab_table, 2)

# Create DataFrame with proper column names
crosstab_df = DataFrame(crosstab_matrix, col_names)
# Add row names as the first column
insertcols!(crosstab_df, 1, :default => row_names)

println("\nAs DataFrame:")
println(crosstab_df)

Crosstab table for default and y:
[7832 1047; 153 10]

As DataFrame:
2×3 DataFrame
 Row │ default  no     yes   
     │ String3  Int64  Int64 
─────┼───────────────────────
   1 │ no        7832   1047
   2 │ yes        153     10


In [7]:
# Remove the temporary stratification column from the original data
DataFrames.select!(data, Not(:strat_var))
nothing

# Feature Engineering

## One hot encoding

In [8]:
# Load the encoder
OneHotEncoder = @load OneHotEncoder pkg=MLJModels

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/frank/.julia/packages/MLJModels/ziReN/src/loading.jl:159


import MLJModels ✔


MLJModels.OneHotEncoder

In [9]:
# Identify categorical columns
categorical_columns = [:job, :marital, :education, :default, :housing, :loan, :contact, :month, :poutcome, :y]

10-element Vector{Symbol}:
 :job
 :marital
 :education
 :default
 :housing
 :loan
 :contact
 :month
 :poutcome
 :y

In [10]:
# Convert string columns to categorical for OneHotEncoder
for col in categorical_columns
    train_data[!, col] = categorical(train_data[!, col])
    test_data[!, col] = categorical(test_data[!, col])
end

In [11]:
# Configure with proper options
encoder = OneHotEncoder(
    drop_last=false,           # Drop reference category
    ordered_factor=false,     # Treat as unordered categorical
    ignore=false             # Set to true to ignore unknown categories
)

# Fit on training data only
cat_train = DataFrames.select(train_data, categorical_columns)
mach = machine(encoder, cat_train)
fit!(mach)

# Transform both datasets
train_encoded = MLJ.transform(mach, cat_train)
test_encoded = MLJ.transform(mach, DataFrames.select(test_data, categorical_columns))

# Remove the y_no column (reference category for target variable)
select!(train_encoded, Not(:y__no))
select!(test_encoded, Not(:y__no))

nothing

┌ Info: Training machine(OneHotEncoder(features = Symbol[], …), …).
└ @ MLJBase /home/frank/.julia/packages/MLJBase/7nGJF/src/machines.jl:499
┌ Info: Spawning 12 sub-features to one-hot encode feature :job.
└ @ MLJModels /home/frank/.julia/packages/MLJModels/ziReN/src/builtins/Transformers.jl:878
┌ Info: Spawning 3 sub-features to one-hot encode feature :marital.
└ @ MLJModels /home/frank/.julia/packages/MLJModels/ziReN/src/builtins/Transformers.jl:878
┌ Info: Spawning 4 sub-features to one-hot encode feature :education.
└ @ MLJModels /home/frank/.julia/packages/MLJModels/ziReN/src/builtins/Transformers.jl:878
┌ Info: Spawning 2 sub-features to one-hot encode feature :default.
└ @ MLJModels /home/frank/.julia/packages/MLJModels/ziReN/src/builtins/Transformers.jl:878
┌ Info: Spawning 2 sub-features to one-hot encode feature :housing.
└ @ MLJModels /home/frank/.julia/packages/MLJModels/ziReN/src/builtins/Transformers.jl:878
┌ Info: Spawning 2 sub-features to one-hot encode feature :loan.

## Standardization for numerical features (TO-DO)


In [12]:
# Get numerical columns (all columns except the categorical ones)
numerical_columns = setdiff(names(train_data), string.(categorical_columns))

#nothing


7-element Vector{String}:
 "age"
 "balance"
 "day"
 "durations"
 "campaign"
 "pdays"
 "previous"

---

# Final sets

In [13]:


# Add numerical features to the encoded datasets
train_final = hcat(train_encoded, DataFrames.select(train_data, numerical_columns))
test_final = hcat(test_encoded, DataFrames.select(test_data, numerical_columns))

nothing

In [14]:
# Create feature matrices and target vectors
X_train = DataFrames.select(train_final, Not(:y__yes))
X_test = DataFrames.select(test_final, Not(:y__yes))

y_train = train_final.y__yes
y_test = test_final.y__yes

nothing

In [15]:
# Convert y_train and y_test to categorical with explicit level ordering
y_train = categorical(y_train, levels=[0, 1])
y_test = categorical(y_test, levels=[0, 1])

nothing

---

# Custom loss function

In [22]:
# Definimos costos de falsos negativos y falsos positivos
cost_FN = 5.0  # Pena mas alta para falsos negativos
cost_FP = 1.0  # Pena mas baja para falsos positivos

# Custom metric: Total cost = (FN * cost_FN) + (FP * cost_FP)
function custom_cost(y_true, y_pred)
    cm = confusion_matrix(y_pred, y_true)
    FN = cm.mat[2, 1]  # True=1, Predicted=0
    FP = cm.mat[1, 2]  # True=0, Predicted=1
    total_cost = FN * cost_FN + FP * cost_FP
    return total_cost
end

custom_cost (generic function with 1 method)

---

# Model

### Random Forest off the shelf

In [16]:
# MLJ way to load the model (resolves ambiguity)
RandomForestClassifier = @load RandomForestClassifier pkg=DecisionTree

rf_model = RandomForestClassifier(
    n_trees=100,
    max_depth=-1,
    rng=123,
    sampling_fraction=0.5,
    min_samples_leaf=5,
    min_samples_split=10
)

# Wrap the model and the training data in a machine.
rf_machine = machine(rf_model, X_train, y_train)

# Train (fit) the model.
MLJ.fit!(rf_machine)

# Make predictions on the test data.
y_pred = MLJ.predict(rf_machine, X_test)

# If you're interested in the predicted labels (instead of probability distributions),
# you can convert them with `mode.`
predicted_labels = mode.(y_pred)

println("Predicted labels: ", predicted_labels)

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/frank/.julia/packages/MLJModels/ziReN/src/loading.jl:159
┌ Info: Training machine(RandomForestClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/frank/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


Predicted labels: CategoricalValue{Float64, UInt32}[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [19]:
# Compute Accuracy
acc = accuracy(predicted_labels, y_test)
println("Accuracy: ", acc)

# Compute the Confusion Matrix
conf_m = confusion_matrix(predicted_labels, y_test);

# Compute F1 Score
f1 = f1score(predicted_labels, y_test)
println("F1 Score: ", f1)

# Compute Recall
rec = recall(predicted_labels, y_test)
println("Recall for class 1: ", rec)  # rec is already the recall for the positive class (1)

Accuracy: 0.9077637690776377
F1 Score: 0.46811224489795916
Recall for class 1: 0.34720908230842007


┌ Warning: Levels not explicitly ordered. Using the order CategoricalValue{Float64, UInt32}[0.0, 1.0]. The "positive" level is 1.0. 
└ @ StatisticalMeasures.ConfusionMatrices /home/frank/.julia/packages/StatisticalMeasures/UTtxb/src/confusion_matrices.jl:339
┌ Warning: Levels not explicitly ordered. Using the order CategoricalValue{Float64, UInt32}[0.0, 1.0]. The "positive" level is 1.0. 
└ @ StatisticalMeasures.ConfusionMatrices /home/frank/.julia/packages/StatisticalMeasures/UTtxb/src/confusion_matrices.jl:339


In [20]:
# Compute the Confusion Matrix
conf_m

          ┌─────────────┐
          │Ground Truth │
┌─────────┼──────┬──────┤
│Predicted│ 0.0  │ 1.0  │
├─────────┼──────┼──────┤
│   0.0   │ 7841 │ 690  │
├─────────┼──────┼──────┤
│   1.0   │ 144  │ 367  │
└─────────┴──────┴──────┘


In [29]:
# Cost

custom_cost(predicted_labels, y_test)

3594.0

### Optimization of hyperparameters RF

In [23]:
using BayesianOptimization

function make_objective(X, y)
    function rf_objective(x)
        # Define param order
        n_estimators     = Int(round(x[1]))
        max_depth        = Int(round(x[2]))
        min_samples_leaf = Int(round(x[3]))
        min_samples_split = Int(round(x[4]))
        sampling_fraction = x[5]  # already Float64

        model = RandomForestClassifier(
            n_trees = n_estimators,
            max_depth = max_depth,
            min_samples_leaf = min_samples_leaf,
            min_samples_split = min_samples_split,
            sampling_fraction = sampling_fraction
        )

        mach = machine(model, X, y)
        res = evaluate!(mach, resampling=CV(nfolds=5), measure=custom_cost, operation=predict_mode)
        
        return -res.measurement[1]  # negate because we are maximizing
    end
    return rf_objective
end

make_objective (generic function with 1 method)

In [24]:
using GaussianProcesses  # Required for MeanConst and SEArd

# Define search space
lowerbounds = [10.0, 2.0, 1.0, 2.0, 0.1]  # min values for each param
upperbounds = [400.0, 50.0, 10.0, 20.0, 1.0]  # max values

# Define GP model
model = ElasticGPE(
    5,
    mean = MeanConst(0.0),
    kernel = SEArd(ones(5), 5.0),
    logNoise = log(1e-4),
    capacity = 3000
)


# Kernel has 6 parameters: 5 ARD + 1 scale:

kernbounds_s = kernbounds_s = [[-2,-2,-2, -2, -2, -5],  # lower bounds for each parameter
                               [2, 2, 2, 2, 2, 5] ] # upper bounds for each parameter
                                 

modeloptimizer = MAPGPOptimizer(
    every = 30,

    noisebounds = [-10, 3],
    kernbounds = kernbounds_s,  # lower and upper bounds for 6 params + mean param
    method = :LN_BOBYQA,  # Local, derivative-free optimizer
    domean = false,  # Do not optimize the mean parameter
    restarts = 5,        # Number of optimization restarts
    maxeval = 100
)

acquisition = ExpectedImprovement()

rf_objective = make_objective(X_train, y_train)

# Build optimizer
opt = BOpt(
    rf_objective,
    model,
    acquisition,
    modeloptimizer,
    lowerbounds,
    upperbounds;
    sense = Max,               
    maxiterations = 50,                
    verbosity = Progress,
    repetitions = 3
)

# Run optimization
result = boptimize!(opt)



┌ Warning: use values(kwargs) and keys(kwargs) instead of kwargs.data and kwargs.itr
│   caller = #MAPGPOptimizer#19 at gp.jl:37 [inlined]
└ @ Core /home/frank/.julia/packages/BayesianOptimization/nq1HY/src/models/gp.jl:37
Evaluating over 5 folds: 100%[=========================] Time: 0:00:05
Evaluating over 5 folds: 100%[=========================] Time: 0:00:02
Evaluating over 5 folds: 100%[=========================] Time: 0:00:03
Evaluating over 5 folds: 100%[=========================] Time: 0:00:36
Evaluating over 5 folds: 100%[=========================] Time: 0:00:35
Evaluating over 5 folds: 100%[=========================] Time: 0:00:37
Evaluating over 5 folds: 100%[=========================] Time: 0:00:29
Evaluating over 5 folds: 100%[=========================] Time: 0:00:31
Evaluating over 5 folds: 100%[=========================] Time: 0:00:30
Evaluating over 5 folds: 100%[=========================] Time: 0:00:03
Evaluating over 5 folds: 100%[=========================] Time: 0:00

(observed_optimum = -2680.1999834107496,
 observed_optimizer = [397.43027116457426, 42.861192065031716, 4.493575091102045, 2.719869024451256, 0.722865842042675],
 model_optimum = -1.1100046699869173e-12,
 model_optimizer = [121.50348727887089, 45.3043103767943, 3.4175885458248185, 19.983088909816075, 0.6411708460922412],)

#### Fit optimized model

In [25]:
params = result.observed_optimizer


n_estimators     = Int(round(params[1]))
max_depth        = Int(round(params[2]))
min_samples_leaf = Int(round(params[3]))
min_samples_split = Int(round(params[4]))
sampling_fraction = params[5]  # already Float64

modelopt = RandomForestClassifier(
    n_trees = n_estimators,
    max_depth = max_depth,
    min_samples_leaf = min_samples_leaf,
    min_samples_split = min_samples_split,
    sampling_fraction = sampling_fraction
)

rf_mach_opt = machine(modelopt, X_train, y_train)

# Train (fit) the model.
MLJ.fit!(rf_mach_opt)

# Make predictions on the test data.
y_pred_opt = MLJ.predict(rf_mach_opt, X_test)

# If you're interested in the predicted labels (instead of probability distributions),
# you can convert them with `mode.`
opt_labels = mode.(y_pred_opt)

┌ Info: Training machine(RandomForestClassifier(max_depth = 43, …), …).
└ @ MLJBase /home/frank/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


9042-element CategoricalArray{Float64,1,UInt32}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

#### Optimized metrics

In [26]:
# Compute Accuracy
acc = accuracy(opt_labels, y_test)
println("Accuracy: ", acc)

# Compute the Confusion Matrix
conf_m_opt = confusion_matrix(opt_labels, y_test);

# Compute F1 Score
f1 = f1score(opt_labels, y_test)
println("F1 Score: ", f1)

# Compute Recall
rec = recall(opt_labels, y_test)
println("Recall for class 1: ", rec[1]) # rec[1] is the recall for class 1

Accuracy: 0.9111922141119222
F1 Score: 0.5323238206173558
Recall for class 1: 0.4323557237464522


┌ Warning: Levels not explicitly ordered. Using the order CategoricalValue{Float64, UInt32}[0.0, 1.0]. The "positive" level is 1.0. 
└ @ StatisticalMeasures.ConfusionMatrices /home/frank/.julia/packages/StatisticalMeasures/UTtxb/src/confusion_matrices.jl:339
┌ Warning: Levels not explicitly ordered. Using the order CategoricalValue{Float64, UInt32}[0.0, 1.0]. The "positive" level is 1.0. 
└ @ StatisticalMeasures.ConfusionMatrices /home/frank/.julia/packages/StatisticalMeasures/UTtxb/src/confusion_matrices.jl:339


In [27]:
conf_m_opt

          ┌─────────────┐
          │Ground Truth │
┌─────────┼──────┬──────┤
│Predicted│ 0.0  │ 1.0  │
├─────────┼──────┼──────┤
│   0.0   │ 7782 │ 600  │
├─────────┼──────┼──────┤
│   1.0   │ 203  │ 457  │
└─────────┴──────┴──────┘


In [28]:
# Cost

custom_cost(opt_labels, y_test)

3203.0